imports

In [45]:
# imports ..
import pandas as pd
import numpy as np
import sklearn 


Read the dataset

In [46]:
data_folder = './data/'
train_file = f'{data_folder}training_v2.csv'

train_df = pd.read_csv(train_file)

train_df.head(5)

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.95,0,Caucasian,F,172.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.64,1,Caucasian,F,165.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,NaN,0,Caucasian,M,188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


Handle nans

In [47]:
# check how many nulls are present
total_na = train_df.isna().sum().sum()
total_null = train_df.isnull().sum().sum()
total_na + total_null

11486984

In [48]:
# remove nulls - for continues values use mean, for discrete use mode

for column in list(train_df.columns):
    value_to_insert = None
    if train_df[column].dtype == float:
        #print(f'float value found! at: {column}')
        value_to_insert = train_df[column].mean()        
    else:
        value_to_insert = train_df[column].mode()[0]
    #print(f'column: {column}, replacing value: {value_to_insert}')
    train_df[column].fillna(value_to_insert, inplace=True)

total_na = train_df.isna().sum().sum()
total_null = train_df.isnull().sum().sum()
assert total_na == 0 and total_null == 0

Data transformation

In [55]:
# get all columns where the values are strings
string_columns = train_df.select_dtypes(include=[object])
string_columns_names = list(string_columns.columns)

# replace these columns with their enumerated value
for column_name in string_columns_names:    
    enumerated_unique_values = list(enumerate(string_columns[column_name].unique()))
    value_to_index_mapper = dict([(y,x) for x,y in enumerated_unique_values])
    train_df[column_name].replace(value_to_index_mapper, inplace=True)

# make sure no string columns are left
assert len(train_df.select_dtypes(include=[object]).columns) == 0